# Capter2 感知机模型
## 感知机模型

In [8]:
import numpy as np
import random
class Model1():
    def fit(self, x, y, epoch, lr):
        self.w = np.zeros(x.shape[1])
        self.b = 0
        self.loss_lst = []
        for i in range(epoch):
            arr = y * (np.dot(self.w, x.T) + self.b)
            loss = -1 * np.where(arr < 0, arr, 0).sum()
            self.loss_lst.append(loss)
            x_err = x[arr<=0]
            y_err = y[arr<=0]
            idx = random.randint(0, x_err.shape[0]-1)
            w_grad = x_err[idx] * y_err[idx] * -1
            b_grad = y_err[idx] * -1
            w_delt = w_grad * lr
            b_delt = b_grad * lr
            self.w = self.w - w_delt
            self.b = self.b - b_delt
    
    def predict(self, x):
        y = np.dot(self.w, x.T) + self.b
        return np.where(y>=0, 1, -1)

## 感知机对偶模型

In [9]:
class Model2():
    def fit(self, x, y, epoch, lr):
        self.a = np.zeros(x.shape[0])
        self.loss_lst = []
        gram_matrix = np.dot(x, x.T)
        self.b = 0
        for i in range(epoch):
            tmp = self.a * y
            for j in range(len(y)):
                if y[j] * (np.dot(tmp,gram_matrix[:, j]) + self.b) <= 0:
                    self.a[j] = self.a[j]+lr
                    self.b = self.b+lr * y[j]
                    break
            loss = y * np.dot(tmp, gram_matrix) + self.b
            loss = -1 * np.where(loss< 0, loss, 0).sum()
            self.loss_lst.append(loss)
        self.w = np.multiply(self.a * y, x.T).sum(axis=1)
    
    def predict(self, x):
        res = np.dot(self.w, x.T) + self.b
        return np.where(res>=0, 1, -1)

## 公共评估函数

In [10]:
def plot_decision_boundary(pre_func, fea, label, ax, title=''):
    fea1_min, fea1_max = fea[:, 0].min() - 0.5, fea[:, 0].max() + 0.5
    fea2_min, fea2_max = fea[:, 1].min() - 0.5, fea[:, 1].max() + 0.5
    s = 0.05
    g_fea1, g_fea2 = np.meshgrid(np.arange(fea1_min, fea1_max, s), np.arange(fea2_min, fea2_max, s))
    pre_y = pre_func(np.c_[g_fea1.ravel(), g_fea2.ravel()])
    pre_y = pre_y.reshape(g_fea1.shape)
    ax.contourf(g_fea1, g_fea2, pre_y, cmap=plt.cm.Spectral)
    ax.scatter(fea[:, 0], fea[:, 1], c = label, s = 20)
    if len(title) > 0:
        ax.set_title(title)

def get_accuracy(gt_y, pre_y):
    diff = np.subtract(gt_y, pre_y)
    err = diff[diff!=0].shape[0]
    return 1 - err/gt_y.shape[0]

## 模型测试
### 准备数据

In [11]:
%matplotlib widget
from sklearn import datasets
import matplotlib.pyplot as plt

sample_num = 200
train_num = 100
inputs, labels = datasets.make_classification(n_samples=sample_num, n_features=2, n_classes=2, n_redundant=0)
labels = np.where(labels == 0, -1, labels)
train_x = inputs[:train_num]
train_y = labels[:train_num]
test_x = inputs[sample_num - train_num:]
test_y = labels[sample_num - train_num:]

### 感知机模型训练和评估

In [12]:
epoch = 15
lr = 0.05
model = Model1()
model.fit(train_x, train_y, epoch, lr)
pre_y = model.predict(test_x)
print('accuracy: %.2f' % get_accuracy(test_y, pre_y))
fig, axs = plt.subplots(1, 2)
axs[0].set_title('loss')
axs[1].set_title('classes')
axs[0].plot(model.loss_lst)
plot_decision_boundary(model.predict, train_x, train_y, axs[1])

accuracy: 0.90


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 感知机对偶模型训练和评估

In [13]:
input_shape = train_x.shape
epoch = 15
lr = 0.05
model = Model2()
model.fit(train_x, train_y, epoch, lr)
pre_y = model.predict(test_x)
print('accuracy: %.2f' % get_accuracy(test_y, pre_y))
fig, axs = plt.subplots(1, 2)
axs[0].set_title('loss')
axs[1].set_title('classes')
axs[0].plot(model.loss_lst)
plot_decision_boundary(model.predict, train_x, train_y, axs[1])

accuracy: 0.94


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …